In [1]:
!pip install detectron2

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [ ]:
import os
import detectron2
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_dataset_train", {}, "path_to_coco_annotations_train.json", "path_to_train_images")
register_coco_instances("my_dataset_val", {}, "path_to_coco_annotations_val.json", "path_to_val_images")

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"))

cfg.MODEL.WEIGHTS = ""

cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 10000  # Promenite na odgovarajući broj iteracija
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80  # Prilagodite broj klasa vašem skupu podataka

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

im = cv2.imread("path_to_your_image.jpg")
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(outputs["panoptic_seg"][0].to("cpu"), outputs["panoptic_seg"][1])
cv2.imshow('Panoptic Segmentation', out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()
